# Création des csv

Le script suivant permet de générer les fichiers csv attendus par Neo4J.

## Importation des librairies

In [283]:
import pandas as pd
import re

## Chargement des données

In [284]:
Olympic_Athlete_Bio = pd.read_csv("../data/Olympic_Athlete_Bio.csv")
Olympic_Athlete_Event_Results = pd.read_csv("../data/Olympic_Athlete_Event_Results.csv")
Olympic_Games_Medal_Tally = pd.read_csv("../data/Olympic_Games_Medal_Tally.csv")
Olympic_Results = pd.read_csv("../data/Olympic_Results.csv")
Olympics_Country = pd.read_csv("../data/Olympics_Country.csv")
Olympics_Games = pd.read_csv("../data/Olympics_Games.csv")
tokyo_2020_tweets = pd.read_csv("../data/tokyo_2020_tweets.csv")

C:\Users\camil\AppData\Local\Temp\ipykernel_4020\108361225.py:7: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,15) have mixed types. Specify dtype option on import or set low_memory=False.
  tokyo_2020_tweets = pd.read_csv("../data/tokyo_2020_tweets.csv")


## Visualisation des headers

In [285]:
print("Olympic_Athlete_Bio\n",Olympic_Athlete_Bio.columns)
print("Olympic_Athlete_Event_Results\n",Olympic_Athlete_Event_Results.columns)
print("Olympic_Games_Medal_Tally\n",Olympic_Games_Medal_Tally.columns)
print("Olympic_Results\n",Olympic_Results.columns)
print("Olympics_Country\n",Olympics_Country.columns)
print("Olympics_Games\n",Olympics_Games.columns)
print("tokyo_2020_tweets\n",tokyo_2020_tweets.columns)

Olympic_Athlete_Bio
 Index(['athlete_id', 'name', 'sex', 'born', 'height', 'weight', 'country',
       'country_noc', 'description', 'special_notes'],
      dtype='object')
Olympic_Athlete_Event_Results
 Index(['edition', 'edition_id', 'country_noc', 'sport', 'event', 'result_id',
       'athlete', 'athlete_id', 'pos', 'medal', 'isTeamSport'],
      dtype='object')
Olympic_Games_Medal_Tally
 Index(['edition', 'edition_id', 'year', 'country', 'country_noc', 'gold',
       'silver', 'bronze', 'total'],
      dtype='object')
Olympic_Results
 Index(['result_id', 'event_title', 'edition', 'edition_id', 'sport',
       'sport_url', 'result_date', 'result_location', 'result_participants',
       'result_format', 'result_detail', 'result_description'],
      dtype='object')
Olympics_Country
 Index(['noc', 'country'], dtype='object')
Olympics_Games
 Index(['edition', 'edition_id', 'edition_url', 'year', 'city',
       'country_flag_url', 'country_noc', 'start_date', 'end_date',
       'competit

## Génération csv...

### ...entités

| Entités | Attributs | Entités | Attributs |
|---------|-----------|---------|-----------|
| COUNTRY |  <ul> <li>*noc* (PK)</li><li>country</li><li>url_flag</li></ul> | CITY | <ul><li> *city_name* (PK)</li></ul> |
| GAMES_EDITION | <ul><li>*edition_id* (PK)</li><li>edition</li><li> year</li><li>start_date</li><li> end_date</li><li>competition_date</li><li>edition_url</li><li> isHeld</li></ul> | EVENT_SESSION | <ul><li> *id_event* (PK)</li><li> result_date</li><li> result_location</li><li> result_format</li><li> result_detail</li><li> result_description</li><li> result_participants</li><li> event_title</li><li> sport_url</li></ul> |
| ATHLETE | <ul><li> *athlete_id* (PK)</li><li> name</li><li> sex</li><li> born</li><li> height</li><li> weight</li><li> description</li><li> special_notes</li></ul> | MEDAL | <ul><li> *value* (PK) avec seulement : `Gold`, `Silver`, `Bronze`</li></ul> |
| SPORT | <ul><li> *sport* (PK)</li></ul> | RESULT | (résultat d’un(e) athlète à une journée d’épreuve)<ul><li> *id_result* (PK)</li><li> pos</li><li> isTeamSport (bool)</li></ul>(tableau des médailles par année et par pays)<ul><li>*id_result* (PK)</li><li> gold</li><li> silver</li><li> bronze</li><li> total</li></ul> |
| DISCIPLINE | <ul><li>*discipline_name* (PK)</li></ul> | TWEET | <ul><li> *id* (PK)</li><li> hashtags</li><li> date</li><li> text</li><li> user_name</li><li> user_location</li><li> user_description</li><li> user_created</li><li> user_followers</li><li> user_friends</li><li> user_favourites</li><li> user_verified</li><li> source</li><li> retweets</li><li> favorites</li><li> is_retweet</li></ul> |

In [286]:
# COUNTRY
country = Olympics_Country[["noc", "country"]].drop_duplicates()
foo = Olympics_Games[['country_noc','country_flag_url']].drop_duplicates()
country = pd.merge(country, foo, how='left', left_on='noc', right_on='country_noc').drop('country_noc', axis=1)

# CITY
city = Olympics_Games[["city"]].drop_duplicates()

# GAMES_EDITION
GamesEdition = Olympics_Games[["edition_id", "edition", "year", "start_date", "end_date", "competition_date", "edition_url", "isHeld"]]

# EVENT_SESSION
EventSession = None # à compléter ultérieurement !!!!!!!!

# ATHLETE
athlete = Olympic_Athlete_Bio[["athlete_id", "name", "sex", "born", "height", "weight", "description", "special_notes"]]

# MEDAL
medal = pd.DataFrame({'value': ['Gold', 'Silver', 'Bronze']})

# SPORT
sport = Olympic_Athlete_Event_Results[["sport"]].drop_duplicates()

# RESULT
result_A = Olympic_Athlete_Event_Results[["pos", "isTeamSport"]]
id_index = len(result_A)+1
id_result = pd.DataFrame({'id_result': range(1, id_index)})
result_A = pd.concat([id_result, result_A.reset_index(drop=True)], axis=1)

result_C = Olympic_Games_Medal_Tally[["gold", "silver", "bronze", "total"]]
id_result = pd.DataFrame({'id_result': range(id_index, id_index + len(result_C))})
result_C = pd.concat([id_result, result_C.reset_index(drop=True)], axis=1)

result = pd.merge(result_A, result_C, how='outer', on='id_result')

# DISCIPLINE
discipline = Olympic_Athlete_Event_Results[["event"]].drop_duplicates().rename(columns={"event": "discipline"})

# TWEET
## il faut formater les colonnes avant de les utiliser
## cf la section Nettoyage des données


# save into csv
country.to_csv("../data/Neo4J/country.csv", index=False)
city.to_csv("../data/Neo4J/city.csv", index=False)
GamesEdition.to_csv("../data/Neo4J/GamesEdition.csv", index=False)
# EventSession.to_csv("../data/Neo4J/EventSession.csv", index=False)
athlete.to_csv("../data/Neo4J/athlete.csv", index=False)
medal.to_csv("../data/Neo4J/medal.csv", index=False)
sport.to_csv("../data/Neo4J/sport.csv", index=False)
result.to_csv("../data/Neo4J/result.csv", index=False)
discipline.to_csv("../data/Neo4J/discipline.csv", index=False)

#### Nettoyage des données

On formate les données issues du csv fourni : *tokyo_2020_tweets.csv*

In [287]:
input = "../data/tokyo_2020_tweets.csv"
transition = "../data/tokyo_2020_tweets_transition.csv"
output = "../data/tokyo_2020_tweets_clean.csv"

# Fonction pour compter les virgules en dehors des guillemets
def count_commas_outside_quotes(line: str) -> int:
    count = 0
    inside_quotes = False
    i = 0
    while i < len(line):
        c = line[i]
        if c == '"':
            # Gestion des guillemets doublés ("")
            if i + 1 < len(line) and line[i + 1] == '"':
                i += 1  # sauter le second guillemet
            else:
                inside_quotes = not inside_quotes  # bascule ouverture/fermeture
        elif c == ',' and not inside_quotes:
            count += 1
        i += 1
    return count

# Première passe : rattacher les lignes incomplètes
with open(input, "r", encoding="utf-8", errors="replace", newline='') as fin, \
         open(transition, "w", encoding="utf-8", newline='') as fout:
    # lire l'en-tête
    header = fin.readline()
    expected_commas = header.count(",")
    fout.write(header)
    for line in fin:
        # compter les virgules qui ne sont pas dans des guillemets      
        comma_count = count_commas_outside_quotes(line)
            
        # on vérifie si la première colonne est un entier
        if not re.match(r'^\s*\d{5,}', line): # on considère que l'ID a au moins 5 chiffres car les âges, années, etc. sont plus petits
            # rattacher à la ligne précédente
            fout.seek(fout.tell() - 1)  # revenir avant le \n
            fout.write("|" + line)
        else:
            # si pas assez de virgules, lire les suivantes
            while comma_count < expected_commas:
                next_line = fin.readline()
                if not next_line:
                    break
                if next_line.strip() == "":
                    continue  # ignorer les lignes vides
                line = line.rstrip("\n\r") + "|" + next_line
                comma_count = line.count(",")
            fout.write(line)


# Le tweet 1419182035810144256 a une incohérence en terme de virgules qui était déjà présente dans le fichier d'origine
# Le tweet 1419519087726706693 aussi et ne possède pas de hashtags
# Après avoir examiné le tweet, on décide de les supprimer
# Le tweet 1419779067730092048 a aussi une incohérence en terme de virgules
# Le tweet 1419932847582990336 aussi
# Après examen, on conserve les tweets (hashtags d'intérêt) mais on corrige manuellement le nombre de virgules
with open(transition, "r", encoding="utf-8") as fin, \
    open(output, "w", encoding="utf-8", newline='') as fout:
    header = fin.readline()
    fout.write(header)
    for line in fin:
        if line.startswith("1419182035810144256,"):
            continue  # sauter cette ligne
        elif line.startswith("1419519087726706693,"):
            continue  # sauter cette ligne
        elif line.startswith("1419779067730092048,"):
            # correction manuelle du tweet
            line = "1419779067730092048,Faizan Lakhani,\"Karachi, Pakistan\",Journalist based in Karachi. Sports Reporter - Geo News.|Tweets are personal opinion.,2010-08-02 16:27:12,231286,1877,11945,True,2021-07-26 21:58:02,All the best to 🇵🇰for their #Tokyo2020 action today. Mahoor Shahzad for her 2nd game in #Badminton against Kirstey… https://t.co/QBqtWIXGnU,\"['Tokyo2020', 'Badminton']\",Twitter for iPhone,7.0,85.0,False\n"
        elif line.startswith("1419932847582990336,"):
            # correction manuelle du tweet
            line = "1419932847582990336,Amith Gosai,Durban,I live my life one Internet post at a time. | Some call me a model whisperer or car blogger. || I'm the guy your mother didn't warn you about.,2009-03-09 11:47:06,3799,574,2202,False,2021-07-27 08:09:06,\"The SA women doing us proud this morning, 2 silver medals in the bag #Tokyo2020\",['Tokyo2020'],TweetDeck,1.0,2.0,False\n"
        fout.write(line)

On charge les données formatées.

In [288]:
tweet = pd.read_csv("../data/tokyo_2020_tweets_clean.csv")

C:\Users\camil\AppData\Local\Temp\ipykernel_4020\1123063261.py:1: DtypeWarning: Columns (8,15) have mixed types. Specify dtype option on import or set low_memory=False.
  tweet = pd.read_csv("../data/tokyo_2020_tweets_clean.csv")


On filtre les lignes qui n'ont pas de hashtags et on modifie les données au format *list* pour les parcourir plus tard.

In [289]:
# on supprime les tweets sans hashtags
tweet = tweet[tweet['hashtags'].notna()]

# on formate les hashtags pour les parcourir
list_hash = []
for hash in tweet['hashtags']:
    new_hash = hash.replace("[", "").replace("]", "").replace("'", "").replace('"', "").split(", ")
    list_hash.append(new_hash)
tweet['hashtags'] = list_hash

On modifie les hashtags passant de *ArtisticGymnastics* à *Artistic Gymnastics*. Ceci afin de pouvoir les comparer plus tard avec la liste des hashtags valides.

In [290]:
# Nettoyage des hashtags pour séparer les majuscules et minuscules collées
pattern = r'(?<=[a-z])(?=[A-Z])' # correspond à une position entre une minuscule et une majuscule

values_hashtag = tweet['hashtags'].tolist()
replace_list = []
list_to_test = []

for element in values_hashtag:
    for hash in element:
        split_hash = re.split(pattern, hash)
        new_hash = ' '.join(split_hash)
        list_to_test.append(new_hash)
    replace_list.append(list_to_test)
    list_to_test = []

tweet['hashtags'] = replace_list

On génère la liste qui contient l'ensemble des valeurs que l'on souhaite conserver parmis les hashtags.

In [291]:


list_hash_valide = []
list_hash_valide.append(athlete["name"].tolist())
list_hash_valide.append(sport["sport"].tolist())

# nettoyage des disciplines pour enlever les sous-disciplines (men, women...)
dscp = discipline["discipline"].tolist()
for element in dscp:
    dscp_clean = element.replace("\"","").split(", ")[0]
    discipline['discipline'] = discipline['discipline'].replace(element, dscp_clean)

disciplineFOO = discipline[["discipline"]].drop_duplicates()
disciplineFOO.to_csv("../data/Neo4J/discipline.csv", index=False)

list_hash_valide.append(discipline["discipline"].tolist())

# on aplatie la liste des listes en un ensemble de hashtags valides
valid_hashes = set(
    h for sublist in list_hash_valide for h in (sublist if isinstance(sublist, list) else [sublist])
)

On récupère les hashtags à tester.

In [292]:
tweet_test = tweet['hashtags']
tweet_test = tweet_test.tolist()
print(tweet_test)

[['Tokyo2020'], ['Tokyo2020'], ['Tokyo2020', 'Artistic Gymnastics', '7Olympics', 'Olympic Games'], ['Tokyo2020', 'Olympics', 'Mirabai Chanu', 'Weightlifting'], ['Tokyo2020', 'CANv GER'], ['Laba Pilipinas', 'Team Philippines', 'Tokyo2020', 'Olympics', 'Tokyo Olympics'], ['Tokyo2020'], ['Tokyo2020', 'Tokyo Olympics'], ['Vikas Krishan'], ['Tokyo2020', 'Tennis', 'Olympics'], ['silvermedal', 'Mirabai Chanu', 'Weightlifting', 'Tokyo2020', 'Tokyo Olympics', 'Team India'], ['Artistic Gymnastics', 'Tokyo2020'], ['Olympics', 'Tokyo2020', 'Olympic Games'], ['Tokyo2020'], ['Tokyo2020', 'Mirabai Chanu', 'Weightlifting'], ['Tokyo2020', 'Olympics'], ['Team GB', 'Hockey', 'Tokyo2020'], ['Saturday Motivation', 'Fenny', 'Today'], ['Mirabai Chanu', 'Tokyo2020', 'Olympics'], ['Tokyo2020'], ['Tokyo2020'], ['Olympic Games', 'Tokyo2020', 'Team India'], ['Tokyo2020', 'Olympics', 'PHI'], ['Team India', 'Tokyo2020', 'Boxing'], ['Tokyo Olympics', 'Tokyo2020', 'Olympic Games', 'Olympics2021', 'Human Rights'], ['M

On filtre les hashtags présents dans tweet.

In [293]:
valid_tweet_indices = []
valid_hashes = set(valid_hashes)

for index, hashtags in enumerate(tweet_test):
    for h in hashtags:
        if h in valid_hashes:
            valid_tweet_indices.append(index)
            break  # passe à la ligne suivante

tweet_valid = tweet.iloc[valid_tweet_indices]
tweet_valid.to_csv("../data/Neo4J/tweet.csv", index=False)

### ...relations

|Nom de la relation | Relation de type Cypher | PK à mettre en relation |
|-------------------|-------------------------|-------------------------|
|REPRESENTS|(:Athlete)-[:REPRESENTS]->(:Country) |id_athlete (Olympic_Athlete_Bio.csv); noc (Olympics_Country.csv)|
|BELONGS_TO|(:City)-[:BELONGS_TO]->(:Country)| City (Olympics_Games.csv); country_noc (Olympics_Games.csv)|
|HOSTS|(:Country)-[:HOSTS]->(:GamesEdition)|country_noc (Olympics_Games.csv); edition_id (Olympics_Games.csv)|
|SCHEDULES|(:GamesEdition)-[:SCHEDULES]->(:EventSession)   | edition_id (Olympic_Athlete_Event_Results.csv); id_event (à ajouter dans Olympic_Athlete_Event_Results.csv)|
|INCLUDES_EVENT|(:Discipline)-[:INCLUDES_EVENT]->(:EventSession)   | event (Olympic_Athlete_Event_Results.csv); id_event (à ajouter dans Olympic_Athlete_Event_Results.csv)|
|TAKES_PLACE_IN|(:GamesEdition)-[:TAKES_PLACE_IN]->(:City)   | edition_id (Olympics_Games.csv); city (Olympics_Games.csv)|
|CONTAINS|(:Sport)-[:CONTAINS]->(:Discipline)   | sport (Olympic_Athlete_Event_Results.csv); event (Olympic_Athlete_Event_Results.csv)|
|PARTICIPATES_IN|(:Country)-[:PARTICIPATES_IN]->(:GamesEdition)   | country_noc (Olympic_Games_Medal_Tally.csv); edition_id (Olympic_Games_Medal_Tally.csv)|
|PLAYS_IN|(:Athlete)-[:PLAYS_IN]->(:EventSession)   | athlete_id (Olympic_Athlete_Event_Results.csv); id_event (à ajouter dans Olympic_Athlete_Event_Results.csv)|
|OWNS_RESULT|(:Country)-[:OWNS_RESULT]->(:Result)   | country_noc (Olympic_Games_Medal_Tally.csv); id_result (à ajouter dans Olympic_Games_Medal_Tally.csv)|
|HAS_RESULT|(:Athlete)-[:HAS_RESULT]->(:Result)   | athlete_id (Olympic_Athlete_Event_Results.csv); id_result (à ajouter dans Olympic_Athlete_Event_Results.csv)|
|AWARDS|(:Result)-[:AWARDS]->(:Medal)   | id_result (à ajouter dans Olympic_Athlete_Event_Results.csv); medal (Olympic_Athlete_Event_Results.csv)|
|MENTIONS_ATHLETE|(:Tweet)-[:MENTIONS_ATHLETE]->(:Athlete)   | id (tokyo_2020_tweets.csv); id_athlete (à ajouter dans new tokyo_2020_tweets.csv)|
|MENTIONS_DISCIPLINE|(:Tweet)-[:MENTIONS_DISCIPLINE]->(:Discipline)   | id (tokyo_2020_tweets.csv); event (à ajouter dans new tokyo_2020_tweets.csv)|
|MENTIONS_SPORT|(:Tweet)-[:MENTIONS_SPORT]->(:Sport)   | id (tokyo_2020_tweets.csv); sport (à ajouter dans new tokyo_2020_tweets.csv)|

In [294]:
country = pd.read_csv("../data/Neo4J/country.csv")
city = pd.read_csv("../data/Neo4J/city.csv")
GamesEdition = pd.read_csv("../data/Neo4J/GamesEdition.csv")
athlete = pd.read_csv("../data/Neo4J/athlete.csv")
medal = pd.read_csv("../data/Neo4J/medal.csv")
sport = pd.read_csv("../data/Neo4J/sport.csv")
result = pd.read_csv("../data/Neo4J/result.csv")
discipline = pd.read_csv("../data/Neo4J/discipline.csv")
tweet = pd.read_csv("../data/Neo4J/tweet.csv")

ensemble_attributs = []
ensemble_attributs.append(country.columns.tolist())
ensemble_attributs.append(city.columns.tolist())
ensemble_attributs.append(GamesEdition.columns.tolist())
ensemble_attributs.append(athlete.columns.tolist())
ensemble_attributs.append(medal.columns.tolist())
ensemble_attributs.append(sport.columns.tolist())
ensemble_attributs.append(result.columns.tolist())
ensemble_attributs.append(discipline.columns.tolist())
ensemble_attributs.append(tweet.columns.tolist())

# vérification des attributs uniques
list_attributs = []
for attribut in ensemble_attributs:
    for h in attribut:
        list_attributs.append(h)
print(len(list_attributs))

list_attributs = pd.Series(list_attributs)
print(len(list_attributs.unique()))

# on aplatie la liste des attributs en un ensemble d'attributs
attributs = set(
    h for sublist in ensemble_attributs for h in (sublist if isinstance(sublist, list) else [sublist])
)

C:\Users\camil\AppData\Local\Temp\ipykernel_4020\2953101235.py:7: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  result = pd.read_csv("../data/Neo4J/result.csv")


46
46


id (tokyo_2020_tweets.csv); event (à ajouter dans new tokyo_2020_tweets.c
id (tokyo_2020_tweets.csv); sport (à ajouter dans new tokyo_2020_tweets.csv)|

In [295]:
# Création de la relation
REPRESENTS = Olympic_Athlete_Bio[["athlete_id", "country_noc"]]
# Vérification des attributs
print(athlete.columns)
print(country.columns)
REPRESENTS.rename(columns={"athlete_id": "athlete_id", "country_noc": "noc"})
# Save into csv
REPRESENTS.to_csv("../data/Neo4J/REPRESENTS.csv", index=False)

Index(['athlete_id', 'name', 'sex', 'born', 'height', 'weight', 'description',
       'special_notes'],
      dtype='object')
Index(['noc', 'country', 'country_flag_url'], dtype='object')


In [296]:
BELONGS_TO = Olympics_Games[["country_noc", "city"]]
print(country.columns)
print(city.columns)
BELONGS_TO.rename(columns={"country_noc": "noc", "city": "city"})
BELONGS_TO.to_csv("../data/Neo4J/BELONGS_TO.csv", index=False)

Index(['noc', 'country', 'country_flag_url'], dtype='object')
Index(['city'], dtype='object')


In [297]:
HOSTS = Olympics_Games[["edition_id", "city"]]
print(GamesEdition.columns)
print(city.columns)
HOSTS.to_csv("../data/Neo4J/HOSTS.csv", index=False)

Index(['edition_id', 'edition', 'year', 'start_date', 'end_date',
       'competition_date', 'edition_url', 'isHeld'],
      dtype='object')
Index(['city'], dtype='object')


In [298]:
# SCHEDULES

# nécessite la création de l'entité EventSession

In [299]:
# nécessite la création de l'entité EventSession
# modification de l'architecture du graphe

# nettoyage des disciplines pour enlever les sous-disciplines (men, women...)
event = Olympic_Athlete_Event_Results[["edition_id", "event", "athlete_id", "sport"]]

dscp = Olympic_Athlete_Event_Results["event"]
dscp = dscp.tolist()
dscp = set(dscp)

for element in dscp:
    dscp_clean = element.split(", ")[0]
    event['event'] = event['event'].replace(element, dscp_clean)

INCLUDES_EVENT = event[["edition_id", "event"]].rename(columns={"edition_id": "edition_id", "event": "discipline"})

print(GamesEdition.columns)
print(discipline.columns)

INCLUDES_EVENT.to_csv("../data/Neo4J/INCLUDES_EVENT.csv", index=False)

C:\Users\camil\AppData\Local\Temp\ipykernel_4020\626780591.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event['event'] = event['event'].replace(element, dscp_clean)


Index(['edition_id', 'edition', 'year', 'start_date', 'end_date',
       'competition_date', 'edition_url', 'isHeld'],
      dtype='object')
Index(['discipline'], dtype='object')


In [300]:
# nécessite la création de l'entité EventSession
# modification de l'architecture du graphe

PLAYS_IN_DISCIPLINE = event[["athlete_id", "event"]].rename(columns={"athlete_id": "athlete_id", "event": "discipline"})
print(athlete.columns)
print(discipline.columns)
PLAYS_IN_DISCIPLINE.to_csv("../data/Neo4J/PLAYS_IN_DISCIPLINE.csv", index=False)

Index(['athlete_id', 'name', 'sex', 'born', 'height', 'weight', 'description',
       'special_notes'],
      dtype='object')
Index(['discipline'], dtype='object')


In [301]:
TAKES_PLACE_IN = Olympics_Games[["edition_id", "city"]]
print(GamesEdition.columns)
print(city.columns)
TAKES_PLACE_IN.to_csv("../data/Neo4J/TAKES_PLACE_IN.csv", index=False)

Index(['edition_id', 'edition', 'year', 'start_date', 'end_date',
       'competition_date', 'edition_url', 'isHeld'],
      dtype='object')
Index(['city'], dtype='object')


In [302]:
CONTAINS = event[["sport", "event"]].rename(columns={"sport": "sport", "event": "discipline"})
print(sport.columns)
print(discipline.columns)
CONTAINS.to_csv("../data/Neo4J/CONTAINS.csv", index=False)

Index(['sport'], dtype='object')
Index(['discipline'], dtype='object')


In [303]:
PARTICIPATES_IN = Olympic_Games_Medal_Tally[["country_noc", "edition_id"]]
print(country.columns)
print(GamesEdition.columns)
PARTICIPATES_IN.rename(columns={"country_noc": "noc", "edition_id": "edition_id"})
PARTICIPATES_IN.to_csv("../data/Neo4J/PARTICIPATES_IN.csv", index=False)

Index(['noc', 'country', 'country_flag_url'], dtype='object')
Index(['edition_id', 'edition', 'year', 'start_date', 'end_date',
       'competition_date', 'edition_url', 'isHeld'],
      dtype='object')


In [343]:
# nécessite la création de l'entité EventSession
# modification de l'architecture du graphe
PLAYS_IN = Olympic_Athlete_Event_Results[["edition_id", "athlete_id"]].drop_duplicates()
print(GamesEdition.columns)
print(athlete.columns)
PLAYS_IN.to_csv("../data/Neo4J/PLAYS_IN.csv", index=False)

Index(['edition_id', 'edition', 'year', 'start_date', 'end_date',
       'competition_date', 'edition_url', 'isHeld'],
      dtype='object')
Index(['athlete_id', 'name', 'sex', 'born', 'height', 'weight', 'description',
       'special_notes'],
      dtype='object')


In [313]:
print(country.columns)
print(result.columns)

noc = Olympic_Games_Medal_Tally["country_noc"]
noc = noc.tolist()

edition = Olympic_Games_Medal_Tally["edition_id"]
edition = edition.tolist()

# on récupère les lignes de Olympic_Games_Medal_Tally
resCountry = result[result['total'].notna()]
indexCountry = resCountry["id_result"].tolist()

OWNS_RESULT = pd.DataFrame({'noc': noc, 'id_result': indexCountry, 'edition_country':edition})
OWNS_RESULT.to_csv("../data/Neo4J/OWNS_RESULT.csv", index=False)

Index(['noc', 'country', 'country_flag_url'], dtype='object')
Index(['id_result', 'pos', 'isTeamSport', 'gold', 'silver', 'bronze', 'total'], dtype='object')


In [314]:
athlete_id = Olympic_Athlete_Event_Results["athlete_id"]
athlete_id = athlete_id.tolist()

resAthlete = result[result['isTeamSport'].notna()]
indexAthlete = resAthlete["id_result"].tolist()

edition = Olympic_Athlete_Event_Results["edition_id"]
edition = edition.tolist()

HAS_RESULT = pd.DataFrame({'athlete_id': athlete_id, 'id_result': indexAthlete, 'edition_athlete':edition})
HAS_RESULT.to_csv("../data/Neo4J/HAS_RESULT.csv", index=False)

In [307]:
medal = Olympic_Athlete_Event_Results["medal"]
medal = medal.tolist()

AWARDS = pd.DataFrame({'id_result': indexAthlete, 'medal': medal})

AWARDS = AWARDS.dropna(subset=['medal'])

AWARDS.to_csv("../data/Neo4J/AWARDS.csv", index=False)

In [308]:
# MENTIONS_ATHLETE
print(athlete.columns)
print(tweet.columns)

# on récupère les athlètes
athlete_names = athlete["name"]
correspondance = pd.DataFrame({'id': athlete["athlete_id"], 'name': athlete["name"]})

# on génère la liste à comparer
list_athlete = athlete_names.tolist()
list_athlete = set(list_athlete)

# on filtre les tweets qui mentionnent au moins un athlète
# on récupère l'id du tweet et le nom des athlètes mentionnés

tweet_test = tweet['hashtags']

id_tweet = []
athlete_hashes = []

for index, hashtags in enumerate(tweet_test):
    hashtags = hashtags.strip("[]").replace("'", "").split(", ")
    for h in hashtags:
        if h in list_athlete:
            id_tweet.append(int(tweet.iloc[index, 0]))  # on récupère l'id du tweet
            athlete_hashes.append(h)

# on associe les id des athlètes aux id des tweets en fonction des noms
MENTIONS_ATHLETE = pd.DataFrame({'id_tweet': id_tweet, 'name': athlete_hashes})
MENTIONS_ATHLETE = pd.merge(MENTIONS_ATHLETE, correspondance, how='left', on='name').drop('name', axis=1)
MENTIONS_ATHLETE = MENTIONS_ATHLETE.rename(columns={'id_tweet':'id', 'id': 'athlete_id'})

MENTIONS_ATHLETE.to_csv("../data/Neo4J/MENTIONS_ATHLETE.csv", index=False)

Index(['athlete_id', 'name', 'sex', 'born', 'height', 'weight', 'description',
       'special_notes'],
      dtype='object')
Index(['id', 'user_name', 'user_location', 'user_description', 'user_created',
       'user_followers', 'user_friends', 'user_favourites', 'user_verified',
       'date', 'text', 'hashtags', 'source', 'retweets', 'favorites',
       'is_retweet'],
      dtype='object')


In [309]:
# MENTIONS_SPORT
print(sport.columns)
print(tweet.columns)

# on récupère les athlètes
list_sport = sport["sport"]

# on génère la liste à comparer
list_sport = list_sport.tolist()
list_sport = set(list_sport)

# on filtre les tweets qui mentionnent au moins un athlète
# on récupère l'id du tweet et le nom des athlètes mentionnés

tweet_test = tweet['hashtags']

id_tweet = []
sport_hashes = []

for index, hashtags in enumerate(tweet_test):
    hashtags = hashtags.strip("[]").replace("'", "").split(", ")
    for h in hashtags:
        if h in list_sport:
            id_tweet.append(int(tweet.iloc[index, 0]))  # on récupère l'id du tweet
            sport_hashes.append(h)

# on associe les id des athlètes aux id des tweets en fonction des noms
MENTIONS_SPORT = pd.DataFrame({'id_tweet': id_tweet, 'sport': sport_hashes})
MENTIONS_SPORT = MENTIONS_SPORT.rename(columns={'id_tweet':'id', 'sport': 'sport'})

MENTIONS_SPORT.to_csv("../data/Neo4J/MENTIONS_SPORT.csv", index=False)

Index(['sport'], dtype='object')
Index(['id', 'user_name', 'user_location', 'user_description', 'user_created',
       'user_followers', 'user_friends', 'user_favourites', 'user_verified',
       'date', 'text', 'hashtags', 'source', 'retweets', 'favorites',
       'is_retweet'],
      dtype='object')


In [310]:
# MENTIONS_DISCIPLINE
print(discipline.columns)
print(tweet.columns)

# on récupère les athlètes
list_discipline = discipline["discipline"]

# on génère la liste à comparer
list_discipline = list_discipline.tolist()
list_discipline = set(list_discipline)

# on filtre les tweets qui mentionnent au moins un athlète
# on récupère l'id du tweet et le nom des athlètes mentionnés

tweet_test = tweet['hashtags']

id_tweet = []
discipline_hashes = []

for index, hashtags in enumerate(tweet_test):
    hashtags = hashtags.strip("[]").replace("'", "").split(", ")
    for h in hashtags:
        if h in list_discipline:
            id_tweet.append(int(tweet.iloc[index, 0]))  # on récupère l'id du tweet
            discipline_hashes.append(h)

# on associe les id des athlètes aux id des tweets en fonction des noms
MENTIONS_DISCIPLINE = pd.DataFrame({'id_tweet': id_tweet, 'discipline': discipline_hashes})
MENTIONS_DISCIPLINE = MENTIONS_DISCIPLINE.rename(columns={'id_tweet':'id', 'discipline': 'discipline'})

MENTIONS_DISCIPLINE.to_csv("../data/Neo4J/MENTIONS_DISCIPLINE.csv", index=False)

Index(['discipline'], dtype='object')
Index(['id', 'user_name', 'user_location', 'user_description', 'user_created',
       'user_followers', 'user_friends', 'user_favourites', 'user_verified',
       'date', 'text', 'hashtags', 'source', 'retweets', 'favorites',
       'is_retweet'],
      dtype='object')


MENTIONS_DISCIPLINE est vide... cela peut être dû à une erreur lors du formatage des données de *tweet*. Nous laissons en attente cette remarque. Nous reviendrons dessus une fois le modèle fonctionnel.